In [15]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as nps
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [16]:
image_names = get_png_names("../data/MEX/")#[0:5]
# image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 200)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

200 municipalities.


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .0001)

butler = geozoom_handler(attn_model, 
                         device, 
                         criterion, 
                         attn_optimizer, 
                         fc_model = fc_model,
                         fc_optimizer = fc_optimizer,
                         fc_batch_size = 4,
                         reduction_percent = .70,
                         max_to_change = 5, 
                         plot = False, 
                         v = False)

In [18]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

print("\nNumber of training images: ", len(train_dl))
print("Number of validation images: ", len(val_dl))

Image convergence dimensions:  (62, 118)

Number of training images:  120
Number of validation images:  80


In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch               Training Loss       Validation Loss     # times clipped     # images at scale   % above max         
Epoch: 0            2159.5928           2157.6398           0                   0                   0                   
Epoch: 1            2152.8167           2152.6634           0                   0                   0                   
Epoch: 2            2147.035            2143.2077           0                   0                   0                   
Epoch: 3            2140.7318           2145.9463           0                   0                   0                   
Epoch: 4            2134.1025           2141.13             0                   0                   0.13333333333333333 
Epoch: 5            2127.5044           2138.0042           0                   0                   0.2916666666666667  
Epoch: 6            2120.531            2130.1758           1                   1                   0.5333333333333333  
Epoch: 7            2112.8258   

In [ ]:
butler.go_dict

In [ ]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape, i)

In [ ]:
# def get_im_dims(im_sizes):
    
#     for muni, size in im_sizes.items():
        
#         last_size = size[-1]
        
#         w, h = last_size[1] - last_size[0], last_size[3] - last_size[2]
        
#         print(muni, w, h)
        
# get_im_dims(butler.image_sizes)

In [ ]:
image_names

In [ ]:
def plot_attn_map(impath):

    """
    Function to save the attention maps for a given image at each threshold
    """

    muni_id = impath.split("/")[3]
    cur_image = load_inputs(impath)
    folder = f"{muni_id}_attention_maps"

    if not os.path.isdir(folder):
        os.mkdir(folder)


    for k in butler.threshold_weights.keys():
        
        print(k)

        if k != 'fc':

            model = butler.attn_model
            model.load_state_dict(butler.threshold_weights[k])
            model.eval()
            IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
            gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda(), target_layer = model.sa) 
            cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)

            fname = os.path.join(f"{muni_id}_attention_maps", f"threshold{k}_muni{muni_id}.png") 

            plot_gradcam(gradcam)
            plt.savefig(fname)
            plt.clf()

In [12]:
butler.plot_attn_map(impath = '../data/MEX/484031098/pngs/484031098_2010_all_box484031098_MAY.png')


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>